In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import regex as re
from values import *

In [ ]:
val = Values()

In [ ]:
artikel_df = pd.read_excel(val.shop_file_path)
marketing_artikel = pd.read_csv(val.marketing_artikel,encoding='latin-1', delimiter=';', on_bad_lines='skip',parse_dates=val.dates,dayfirst=True)

In [ ]:
warengrps = marketing_artikel[['NUMMER','WARENGR']]
# warengrps['WARENGR'] = warengrps['WARENGR'].astype(float).astype(int)
warengrps['WARENGR'] = pd.to_numeric(warengrps['WARENGR'],errors='coerce')
warengrps.dropna(subset='WARENGR',inplace=True)
warengrps['WARENGR'] = warengrps['WARENGR'].astype(int)

In [ ]:
warengrps.drop_duplicates(subset='NUMMER',inplace=True)

In [ ]:
artikels = artikel_df[['StoreId','Name','Beschreibung','Price', 'Var_Text','Farbe','Zutatenliste']]
# artikels.drop_duplicates(subset='ProdIndex',inplace=True)

In [ ]:
artikels['Number'] = artikels['StoreId'].str.split().str[0]


In [ ]:
artikels_mit_wrgp = pd.merge(artikels,warengrps,how='left',left_on='Number',right_on='NUMMER')
artikels_mit_wrgp

In [ ]:
artikels_mit_wrgp = artikels_mit_wrgp[artikels_mit_wrgp['NUMMER'].isna()==False]

In [ ]:
artikels_mit_wrgp['WARENGR'] = artikels_mit_wrgp['WARENGR'].astype(int)

In [ ]:
kuchen_gerate = artikels_mit_wrgp[artikels_mit_wrgp['WARENGR'].isin(val.kuchengerate_wr_list)]
kuchen_gerate = kuchen_gerate[['WARENGR','NUMMER','Name','Beschreibung','Price','Var_Text']]
kuchen_gerate = kuchen_gerate.rename(columns={'Name':'NAME','Beschreibung':'BESCHREIBUNG','Price':'PRICE','Var_Text':'VAR_TEXT'})

In [ ]:
kuchen_gerate['VAR_TEXT'] = kuchen_gerate['VAR_TEXT'].str.replace(' ',' ',regex=True)

In [ ]:
kuchen_gerate.info()

In [ ]:
for id,item in enumerate(kuchen_gerate['BESCHREIBUNG']):
    kuchen_gerate = kuchen_gerate.dropna(subset='BESCHREIBUNG')


In [ ]:
kuchen_gerate['BESCHREIBUNG']

In [ ]:
kuchen_gerate['BESCHREIBUNG'] = kuchen_gerate['BESCHREIBUNG'].str.replace(r'<p>|</p>|<ul>|</ul>|<li>|</li>|<br>|<b>|</b>|<span.*>|<font.*>|<strong>|</strong>',' ',regex=True,case=False)
list(kuchen_gerate[kuchen_gerate['BESCHREIBUNG'].str.contains('<')]['BESCHREIBUNG'])

### Cleaning unwanted characters and rows

In [ ]:
kuchen_gerate['BESCHREIBUNG'] = kuchen_gerate['BESCHREIBUNG'].str.replace('&nbsp_',' ')
kuchen_gerate['BESCHREIBUNG'] =kuchen_gerate['BESCHREIBUNG'].str.replace('&Oslash',' ')
kuchen_gerate['BESCHREIBUNG'] =kuchen_gerate['BESCHREIBUNG'].str.replace('<br>',' ')

# kuchen_gerate = kuchen_gerate[kuchen_gerate['WM'].isna() == True]
kuchen_gerate = kuchen_gerate[kuchen_gerate['NUMMER'].str.match(r'^\w+S$') == False]
kuchen_gerate = kuchen_gerate[kuchen_gerate['NUMMER'].str.match(r'^\w+P$') == False]


In [ ]:
## creating a copy of the original text, to clean it up with the extracted patterns
mined_text = kuchen_gerate[['NUMMER','BESCHREIBUNG']].copy()
mined_text['REMAINED_TEXT'] = mined_text['BESCHREIBUNG'].copy()

In [ ]:
## Extracting sizes from text
mined_text.loc[mined_text['BESCHREIBUNG'].str.contains(r' klein *',case=False),'GROESSE'] = 'S'
mined_text.loc[mined_text['BESCHREIBUNG'].str.contains(r' *groß *|großer* *',case=False),'GROESSE'] = 'L'
mined_text.loc[mined_text['BESCHREIBUNG'].str.contains(r' mittel *',case=False),'GROESSE'] = 'M'
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r' *klein *| groß *| mittel *|großer* *','',regex=True,case=False)

In [ ]:
## Extracting colors
mined_text['FARBE'] = mined_text['BESCHREIBUNG'].str.extract(r'(kirschrot|ofenrot|rot|orange|gelb|blau|schwarz|weiß|weiss|grün|silber|creme|grau|blu |dijongelb| \
                                                         |Elfenbein|gold|aubergine|Dunkelgrün|dunkel- braun|braun)',flags=re.IGNORECASE)
mined_text['FARBE'] = mined_text['FARBE'].str.capitalize()
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'kirschrot|ofenrot|rot|orange|gelb|blau|schwarz|weiß|weiss|grün|silber|creme|grau|blu |dijongelb|Elfenbein|gold| \
                                                                |aubergine|aubergine|Dunkelgrün|dunkel- braun|braun','',regex=True, case=False)

In [ ]:
## Extracting number of items
mined_text['STUECK'] = mined_text['BESCHREIBUNG'].str.extract(r'( \d+ Stück)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r' \d+ Stück','',regex=True, case=False)

In [ ]:
## Extracting number of items in sets
mined_text['TEILIG'] = mined_text['BESCHREIBUNG'].str.extract(r'( *\d+-tlg.|\d+-teilig|\d+tlg.*|\d+-*er set|\d+ set|\d+ ?ply)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r' *\d+-tlg.|\d+-* *teilig[er]*|\d+tlg.*|\d+-*er set|\d+ set|\d+ ?ply','',regex=True, case=False)

mined_text['TEILIG'] = mined_text['TEILIG'].str.replace(r'-tlg.|-tlg |tlg.|-teilig|tlg *',' tlg.',regex=True,case=False)
mined_text['TEILIG'] = mined_text['TEILIG'].str.lstrip()

In [ ]:
## Extracting weight patterns
mined_text['GEWICHT'] = mined_text['BESCHREIBUNG'].str.extract(r'([\d,]*\d+ *kg|\d+ *g |\d+ *gram)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'[\d,]*\d+ *kg|\d+ *g |\d+ *gram','',regex=True,case=False)

In [ ]:
## Extracting height patterns
mined_text['HOHE'] = mined_text['BESCHREIBUNG'].str.extract(r'(\d+ *cm hoch|Pfanne hoch \(\d+ cm\)|\d+,\d+ *cm hoch|hoch \d+ *cm|\d+,*\d* cm hoch|hoch [Ø|ø] \d+ *cm)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'\d+ *cm hoch|Pfanne hoch \(\d+ cm\)|\d+,\d+ *cm hoch|hoch \d+ *cm|\d+,*\d* cm hoch|hoch [Ø|ø] \d+ *cm','',regex=True, case=False)

## cleaning up the height formats
mined_text['HOHE'] = mined_text['HOHE'].str.replace('cm',' cm')
mined_text['HOHE'] = mined_text['HOHE'].str.replace('  cm',' cm')

In [ ]:
### Extracting floor sizes
mined_text['BODEN'] = mined_text['BESCHREIBUNG'].str.extract(r'(Boden [Ø |_]*\d+[,\d]* cm|Boden [Ø |_]*\d+[,\d]* mm| \
                                                         Bodendurchmesser [Ø |_]*\d+[,\d]* cm|Boden-Kontaktfläche [Ø |_]*\d+[,\d]* cm)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'Boden [Ø |_]*\d+[,\d]* cm|Boden [Ø |_]*\d+[,\d]* mm|Bodendurchmesser [Ø |_]*\d+[,\d]* cm| \
                                                                  Boden-Kontaktfläche [Ø |_]*\d+[,\d]* cm','',regex=True, case=False)

## Cleaning up the boden formats
mined_text['BODEN'] = mined_text['BODEN'].str.replace(' Ø','')
mined_text['BODEN'] = mined_text['BODEN'].str.replace(' ø','')

In [ ]:
## Extracting info about the Deckel
mined_text['DECKEL'] = mined_text['BESCHREIBUNG'].str.extract(r'(mit deckel|ohne deckel|mit glasdeckel|m.Deckel|\+Deckel|\+ Deckel|\(Inkl. Deckel\))',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'mit deckel|ohne deckel|mit glasdeckel|m.Deckel|\+Deckel|\+ Deckel|\(Inkl. Deckel\)','',regex=True, case=False)

In [ ]:
## Extracting shape info
mined_text['FORM'] = mined_text['BESCHREIBUNG'].str.extract(r'(rund|oval[er]*|rechteckige|eckige|rechteckig|eckig|quadratisch)',flags=re.IGNORECASE)
mined_text['FORM'] = mined_text['FORM'].str.capitalize()
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'rund|oval[er]*|rechteckige|eckige|rechteckig|eckig|quadratisch','',regex=True, case=False)

In [ ]:
## Extracting info about the griff
mined_text['GRIFF'] = mined_text['BESCHREIBUNG'].str.extract(r'(Gußeisen-* *griffe*n*|Edelstahl-* *griffe|Edelst.-Griffe*n*|Edelstahlgriffe*n*|Gusseisen-* *griffe*n*|EDEL- STAHLgriffe| \
                                                         |Bronze-? ?griffe?n?|Bronzegriffe*n*|Bronze Griffe*n*|mit griffe*n*|griffe*n*|mit \w+ *griffe*)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'Gußeisen-* *griffe*n*|Edelstahl-* *griffe*n*|Edelst.-Griffe*n*|Edelstahlgriffe*n*|Gusseisen-* *griffe*n*|EDEL- STAHLgriffe| \
                                                         |Bronze-? ?griffe?n?|Bronzegriffe*n*|Bronze Griffe*n*|mit griffe*n*|griffe*n*|mit \w+ *griffe*n*','',regex=True, case=False)


### Separating dimensions into separate columns

In [ ]:
## Extracting the patterns related to size and size units and uniforming the formatting
mined_text['MASSEN'] = mined_text['BESCHREIBUNG'].str.extract(r'([\d,*\d* x]*[ *\d,*\d* x]*[ *\d,*\d*] *cm| \d+mm| \d+ mm| \d+cm| \d+mm|\d+ cm)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'([\d+,*\d* x]*[ *\d+,*\d* x]*[ *\d+,*\d*] *cm| \d+mm| \d+ mm| \d+cm| \d+mm|\d+ cm)','',regex=True,case=False)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace(r'^, ','',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace(r'^ *','',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('X','x',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('x',' x ',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('  x  ',' x ',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('cm',' cm',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('  cm',' cm',regex=True)
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace('mm',' mm',regex=True)

## Separating the measurement unit from the values
mined_text['MASSEN_EINHEIT'] = mined_text['MASSEN'].str.extract(r'(cm|mm)')
mined_text['MASSEN'] = mined_text['MASSEN'].str.replace(r'[cm|mm]','',regex=True,case=False)

In [ ]:
## Separating dimensions from one another when there are more than 1
text = mined_text['MASSEN'].str.split(r'x|, ')

df = pd.DataFrame(text)

df['MASSEN_1'] = ''
df['MASSEN_2'] = ''
df['MASSEN_3'] = ''

for index, row in df.iterrows():
    numbers = row['MASSEN']
    if isinstance(numbers, list):
        if len(numbers) == 1:
            df.at[index, 'MASSEN_1'] = numbers[0]
        elif len(numbers) == 2:
            df.at[index, 'MASSEN_1'] = numbers[0]
            df.at[index, 'MASSEN_2'] = numbers[1]
        elif len(numbers) == 3:
            df.at[index, 'MASSEN_1'] = numbers[0]
            df.at[index, 'MASSEN_2'] = numbers[1]
            df.at[index, 'MASSEN_3'] = numbers[2]

df.drop(columns=['MASSEN'], inplace=True)

mined_text['MASSEN_1'] = df['MASSEN_1']
mined_text['MASSEN_2'] = df['MASSEN_2']
mined_text['MASSEN_3'] = df['MASSEN_3']

### Cleaning up the values in columns (making them uniform)

In [ ]:
## Extracting volume and size information and uniforming the formatting
mined_text['VOLUME'] = mined_text['BESCHREIBUNG'].str.extract(r'( *\d+ ml| *\d+l| *\d*,*\d+ *l+[iter]*| *\d+ l)',flags=re.IGNORECASE)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r' *\d+ ml| *\d+l| *\d*,*\d+ *l+[iter]*| *\d+ l','',regex=True,case=False)

mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'L',' L',regex=True)
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'Liter',' L',regex=True,case=False)
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'l',' L',regex=True)

mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'  L',' L',regex=True)
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'm L','mL',regex=True)

## Separating the measuring unit from the values
mined_text['VOLUME_EINHEIT'] = mined_text['VOLUME'].str.extract(r'(L|mL)')
mined_text['VOLUME'] = mined_text['VOLUME'].str.replace(r'[L|mL]','',regex=True)


In [ ]:
## Extracting materials (so many variations that can be for different parts of the item)
new_data = mined_text['BESCHREIBUNG'].str.extractall(r'(steinzeug|granit|edelstahl|ohne keramik|Guss-Aluminium|Gussaluminium|Guß-* *eisen|Anti-* *haftbeschichtung|titan|kupfer| \
                                                 edelst.|keramik|Gußeisen|Gusseisen|Guss|eisen|holz|porzellan|m. Antihaftversiegelung|Guß- alu|Edelst.)',flags=re.IGNORECASE).groupby(level=0).agg(','.join)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'(steinzeug|granit|edelstahl|ohne keramik|Guss-Aluminium|Gussaluminium|Guß-* *eisen|Anti- haftbeschichtung|titan|kupfer| \
                                                                  edelst.|keramik|Gußeisen|Gusseisen|Guss|eisen|holz|porzellan|m. Antihaftversiegelung|Guß- alu|Edelst.)','',regex=True,case=False)

df_l = pd.DataFrame(new_data)
new_df = df_l[0].str.split(',')


In [ ]:
## Transferring material data into separate columns (the data model needs rethinking)
df = pd.DataFrame(new_df)

df['MATERIAL_1'] = ''
df['MATERIAL_2'] = ''
df['MATERIAL_3'] = ''


for index, row in df.iterrows():
    numbers = row[0]
    if isinstance(numbers, list):
        if len(numbers) == 1:
            df.at[index, 'MATERIAL_1'] = numbers[0]
        elif len(numbers) == 2:
            df.at[index, 'MATERIAL_1'] = numbers[0]
            df.at[index, 'MATERIAL_2'] = numbers[1]
        elif len(numbers) == 3:
            df.at[index, 'MATERIAL_1'] = numbers[0]
            df.at[index, 'MATERIAL_2'] = numbers[1]
            df.at[index, 'MATERIAL_3'] = numbers[2]

## Transferring the cleaned data to the original dataframe

mined_text['MATERIAL_1'] = df['MATERIAL_1']
mined_text['MATERIAL_2'] = df['MATERIAL_2']
mined_text['MATERIAL_3'] = df['MATERIAL_3']

## Formatting (Capitalizing) the material items

mined_text['MATERIAL_1'] = mined_text['MATERIAL_1'].str.capitalize()
mined_text['MATERIAL_2'] = mined_text['MATERIAL_2'].str.capitalize()
mined_text['MATERIAL_3'] = mined_text['MATERIAL_3'].str.capitalize()



In [ ]:
## Reordering the columns
mined_text = mined_text[['NUMMER', 'BESCHREIBUNG', 'REMAINED_TEXT','GROESSE', 'FARBE', 'VOLUME','VOLUME_EINHEIT', 'MASSEN_1','MASSEN_2','MASSEN_3', 'MASSEN_EINHEIT', 'STUECK', 'TEILIG', 
      'GRIFF','GEWICHT', 'HOHE', 'BODEN', 'DECKEL', 'FORM','MATERIAL_1','MATERIAL_2','MATERIAL_3']]

In [ ]:
## Cleaning up the Remaining text (The original product name) from irrelevant characters
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'Ø|ø|\(\)|\d+|inhalt|wiegt ?[ca.]*|\.|\+|\/|farbe','',regex=True,case=False)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r',|-|:',' ',regex=True,case=False)
## Removing some problematic cases that couldn't be included in the patterns
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'X X','',regex=True,case=False)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'\(\)','',regex=True,case=False)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'einzeln?|gesamt| personen| xx | z ','',regex=True,case=False)

mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'mit .*','',regex=True,case=False)

mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'  ',' ',regex=True,case=False)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r'  ',' ',regex=True,case=False)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.lstrip()
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.rstrip()
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.replace(r' in$| je lang$','',regex=True,case=False)
mined_text['REMAINED_TEXT'] = mined_text['REMAINED_TEXT'].str.title()


In [ ]:
mined_text[mined_text['REMAINED_TEXT'].str.contains(r' je lang$',case=False)]

In [ ]:
mined_text[mined_text['REMAINED_TEXT'].str.contains('X X')]

In [ ]:
## Exporting data into Excel
mined_text.to_excel('Exports/kuchen_gerate_mined_sample_1.xlsx')